References:
- [Tensorflow Mechanics 101](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/tf/index.html)
- [Running Graphs](https://www.tensorflow.org/versions/r0.9/api_docs/python/client.html)
- [Building Graphs](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html)


In [1]:
from IPython.display import display
import tensorflow as tf

In [2]:
x = tf.constant(3.0)
y = tf.constant(4.0)
z = x + y

print 'z ==', z
print 'z.graph ==', z.graph
print 'x.graph == z.graph: ', z.graph == y.graph
print 'x.graph == default_graph(): ', x.graph == tf.get_default_graph()
sess = tf.Session()
# 7.0
print 'run(z) ==', sess.run(z)
# 7.0
print 'run(z) ==', sess.run(z)
sess.close()

# Error
# print 'run(z) ==', sess.run(z)

z == Tensor("add:0", shape=(), dtype=float32)
z.graph == <tensorflow.python.framework.ops.Graph object at 0x10d8a6c50>
x.graph == z.graph:  True
x.graph == default_graph():  True
run(z) == 7.0
run(z) == 7.0


In [3]:
# Call close() automatically by with-statement.
# Also, with-statement calls as_default() implicitly.
with tf.Session() as sess:
  print 'run(z) ==', sess.run(z)
  print 'z.eval() ==', z.eval()

run(z) == 7.0
z.eval() == 7.0


In [4]:
sess = tf.Session()

# This is error.
# z.eval()
print 'get_default_session', tf.get_default_session()

with sess.as_default():
    print 'get_default_session', tf.get_default_session(), tf.get_default_session() == sess
    print 'z.eval() ==', z.eval()

get_default_session None
get_default_session <tensorflow.python.client.session.Session object at 0x10d8a6610> True
z.eval() == 7.0


In [5]:
x = tf.Variable(3.5)
y = x - tf.constant(4.0)
step = tf.train.GradientDescentOptimizer(0.1).minimize(tf.abs(y))
print step

print '===== default session ====='
with tf.Session():
    # FailedPreconditionError: Attempting to use uninitialized value 
    # x.eval()
    init = tf.initialize_all_variables()
    init.run()
    for i in xrange(10):
        step.run()
        print x.eval()

print '===== explicit session ====='
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
for i in xrange(10):
    sess.run(step)
    print sess.run(x)  # sess does not have 'eval'. Use run.
sess.close()

print '===== explicit session as arg====='
sess = tf.Session()
init = tf.initialize_all_variables()
init.run(session=sess)
for i in xrange(10):
    step.run(session=sess)
    print x.eval(session=sess)
sess.close()

name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"

===== default session =====
3.6
3.7
3.8
3.9
4.0
4.1
4.0
4.1
4.0
4.1
===== explicit session =====
3.6
3.7
3.8
3.9
4.0
4.1
4.0
4.1
4.0
4.1
===== explicit session as arg=====
3.6
3.7
3.8
3.9
4.0
4.1
4.0
4.1
4.0
4.1
